In [ ]:
import os
import cv2
import numpy as np
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

import math
import random
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [ ]:
# --- Configuration ---
DATA_DIR = './data'
N_CLUSTERS = 10000    # BoVW 词典大小
MAX_KP = 100         # 每张图最大关键点数
PATCH_SIZE = 16     # 描述子 patch 大小
INTEREST_METHOD = 'DoG'  # 可选: 'DoG', 'Harris', 'LoG'
EPOCHS = 50
LR = 1e-3

In [ ]:
# --- Setup MPS device ---
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [ ]:
# --- 1. Interest Point Detection ---
def detect_interest_points(gray, method='DoG', threshold=5):
    if method == 'DoG':
        # Define multiple sigma values for Gaussian blur
        num_octaves=3 # the level of the image pyramid
        scales_per_octave=3 # the number of scales per octave
        sigma0=1.6 # otiginal sigma value
        
        keypoints = []
        k = 2 ** (1 / scales_per_octave)  # 每层之间的尺度倍增因子
        
        # form the 0th octave to num_octaves-1 octaves
        for octave in range(num_octaves):
            # 1) create the base image for the current octave
            if octave == 0:
                base = gray.copy()
            else:
                # 下采样前一个 octave 的 base
                base = cv2.pyrDown(prev_base)
            
            # save the current base for the next octave
            prev_base = base
            # calculate the size of the base image
            h, w = base.shape
            
            # 2) 构建本 octave 的 Gaussian 金字塔
            #    需要做 (scales_per_octave + 3) 层，以便产生 scales_per_octave+2 个 DoG 层
            sigmas = [sigma0 * (k ** (i + octave*scales_per_octave)) for i in range(scales_per_octave + 3)]
            gaussians = [cv2.GaussianBlur(base, (0,0), sigmaX=s) for s in sigmas]
            
            # 3) 计算 DoG 图层
            dogs = []
            for i in range(len(gaussians)-1):
                diff = gaussians[i+1].astype(np.float32) - gaussians[i].astype(np.float32)
                dogs.append(diff)
            
            # 4) 在空间 + 尺度域寻找极值点
            #    只从第 1 层到倒数第 2 层做比较，因为要用前后层做上下尺度比较
            for i in range(1, len(dogs)-1):
                # the three layers to compare
                prev_d, curr_d, next_d = dogs[i-1], dogs[i], dogs[i+1]

                # max_spat and min_spat are masks of the same size as curr_d
                # 空间极值：用形态学膨胀/腐蚀检测局部极大/极小值
                # 对 curr_d 做形态学膨胀（dilation），相当于每个像素点被它 3×3 邻域内的最大值取代。
                #curr_d 在每个像素的 3×3 邻域内取最大值，输出也是一个 (H, W) 的数组——max_spat[y, x] 就是 curr_d 在 (y, x) 周围那 9 个点里的最大值。
                max_spat = cv2.dilate(curr_d, np.ones((3,3),np.uint8))
                # 做腐蚀（erosion），相当于每个像素点被邻域内的最小值取代。
                min_spat = cv2.erode( curr_d, np.ones((3,3),np.uint8))
                
                # 跨尺度极值：要比前一 DoG 和后一 DoG 都大（或都小）
                mask_max = (curr_d == max_spat) & (curr_d > prev_d) & (curr_d > next_d) & (np.abs(curr_d) > threshold)
                mask_min = (curr_d == min_spat) & (curr_d < prev_d) & (curr_d < next_d) & (np.abs(curr_d) > threshold)
                
                # Find all coordinates of the points that satisfy the conditions
                coords = np.vstack((np.argwhere(mask_max), np.argwhere(mask_min)))

                # 5) 将坐标映射回原图尺度并创建 KeyPoint
                for y, x in coords:
                    # 因为后面可能 downsample 了，需要乘回 2**octave
                    scale = (2 ** octave)

                    kp = cv2.KeyPoint(float(x*scale), float(y*scale), size=PATCH_SIZE * scale)
                    keypoints.append(kp)
                    
        # 最后按响应或数量截断
        keypoints = sorted(keypoints, key=lambda kp: kp.response if hasattr(kp, 'response') else 1.0,
                        reverse=True)
        return keypoints[:MAX_KP]
    elif method == 'Harris':
        harris = cv2.cornerHarris(gray.astype(np.float32), 2, 3, 0.04)
        coords = np.argwhere(harris > threshold * harris.max())
    elif method == 'LoG':
        log = cv2.Laplacian(gray, cv2.CV_64F)
        coords = np.argwhere(np.abs(log) > threshold)
    else:
        sift = cv2.SIFT_create()
        kps = open_kps = sift.detect(gray, None)
        # raise ValueError('Unsupported method')
    # Create keypoints and limit count
    return kps[:MAX_KP]

In [ ]:
# --- 2. Compute Custom SIFT-like Descriptor ---
def compute_custom_sift_descriptor(gray, keypoints, patch_size=PATCH_SIZE):
    # Compute gradients once
    grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(grad_x, grad_y)
    ori = cv2.phase(grad_x, grad_y, angleInDegrees=True)

    descriptors = []
    half = patch_size // 2
    h, w = gray.shape

    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        # boundary check
        if x-half < 0 or y-half < 0 or x+half >= w or y+half >= h:
            continue
        # extract full patch
        mag_full = mag[y-half:y+half, x-half:x+half]
        ori_full = ori[y-half:y+half, x-half:x+half]
        # 1) Orientation assignment: build 36-bin histogram over full patch
        hist8, bin_edges = np.histogram(
            ori_full.ravel(), bins=8, range=(0,360), weights=mag_full.ravel()
        )
        # main orientation is center of max bin
        max_idx = np.argmax(hist8)
        main_ori = (bin_edges[max_idx] + bin_edges[max_idx+1]) / 2.0
        # normalize full patch orientations relative to main orientation
        ori_full = (ori_full - main_ori + 360) % 360

        # slice into subregions and build descriptor
        desc = []
        for i in range(4):
            for j in range(4):
                sub_mag = mag_full[i*4:(i+1)*4, j*4:(j+1)*4].ravel()
                sub_ori = ori_full[i*4:(i+1)*4, j*4:(j+1)*4].ravel()
                # 2) histogram on aligned orientations
                hsub, _ = np.histogram(sub_ori, bins=8, range=(0,360), weights=sub_mag)
                desc.extend(hsub)
        desc = np.array(desc, dtype=np.float32)
        desc /= (np.linalg.norm(desc) + 1e-7)
        descriptors.append(desc)

    return np.array(descriptors) if descriptors else None

In [ ]:
# --- 3. Load CIFAR-10 ---
# transform = transforms.Compose([transforms.ToTensor()])
transform = transforms.Compose([
    # 先把 32×32 放大到 128×128
    transforms.Resize((128, 128), interpolation=InterpolationMode.BICUBIC),

    # # 再做常见的数据增强（可按需打开/调整顺序）
    # transforms.RandomCrop(120, padding=4),           # 随机裁剪到 120×120 并 pad
    # transforms.RandomHorizontalFlip(p=0.5),          # 随机水平翻转
    # transforms.ColorJitter(brightness=0.2,
    #                        contrast=0.2,
    #                        saturation=0.2,
    #                        hue=0.1),                  # 随机色相／亮度／对比度扰动

    # 转成 Tensor 并归一化到 [0,1]
    transforms.ToTensor(),

    # 再做标准化（CIFAR-10 通常用以下均值/方差）
    # transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
    #                      std =[0.2470, 0.2435, 0.2616]),
])

trainset = CIFAR10(DATA_DIR, train=True, download=True, transform=transform)
testset  = CIFAR10(DATA_DIR, train=False, download=True, transform=transform)

In [ ]:
# 3. 准备逆标准化，以便可视化
# mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3,1,1)
# std  = torch.tensor([0.2470, 0.2435, 0.2616]).view(3,1,1)
# inv_norm = transforms.Normalize((-mean/std).squeeze(), (1/std).squeeze())

indices = random.sample(range(len(trainset)), 10)
imgs = []
for idx in indices:
    img, _ = trainset[idx]      # 返回 (C,H,W) tensor
    # img = inv_norm(img)        # 逆归一化到 [0,1]
    # img = torch.clamp(img, 0, 1)
    imgs.append(img)

grid = make_grid(imgs, nrow=5, padding=2)
plt.figure(figsize=(12, 6))
plt.imshow(grid.permute(1, 2, 0))
plt.axis('off')
plt.show()

In [ ]:
# --- 4. Extract descriptors from training set ---
all_desc = []
img_desc_idx = []
train_labels = []
print('Extracting training descriptors...')
for img, label in tqdm(trainset, desc='Train images'):
    img_np = (img.numpy().transpose(1,2,0) * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    kps = detect_interest_points(gray, method=INTEREST_METHOD)
    des = compute_custom_sift_descriptor(gray, kps)
    if des is not None:
        all_desc.extend(des)
    img_desc_idx.append(len(all_desc))
    train_labels.append(label)
all_desc = np.vstack(all_desc)

In [ ]:
# --- 5. Build visual vocabulary ---
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, batch_size=1000, verbose=1)
kmeans.fit(all_desc)

In [ ]:
# --- 6. Build BoVW features for train ---
def build_bovw(descriptors):
    words = kmeans.predict(descriptors)
    hist, _ = np.histogram(words, bins=np.arange(N_CLUSTERS+1))
    return hist.astype(float) / (hist.sum() + 1e-7)

train_feats = []
start = 0
# for idx in img_desc_idx:
for idx in tqdm(img_desc_idx, desc='Building BoVW features'):
    end = idx
    if end - start > 0:
        train_feats.append(build_bovw(all_desc[start:end]))
    else:
        train_feats.append(np.zeros(N_CLUSTERS, dtype=float))
    start = end
train_feats = np.array(train_feats)
train_labels = np.array(train_labels)

In [ ]:
# --- 7. Extract BoVW features for test set ---
test_feats = []
test_labels = []
print('Extracting test BoVW features...')
for img, label in tqdm(testset, desc='Test images'):
    img_np = (img.numpy().transpose(1,2,0) * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    kps = detect_interest_points(gray, method=INTEREST_METHOD)
    des = compute_custom_sift_descriptor(gray, kps)
    if des is not None:
        hist = build_bovw(des)
    else:
        hist = np.zeros(N_CLUSTERS, dtype=float)
    test_feats.append(hist)
    test_labels.append(label)

test_feats = np.array(test_feats)
test_labels = np.array(test_labels)

## Softmax

In [ ]:
# # --- 8. Train Softmax Regression (Logistic Regression) on MPS ---
# # Prepare PyTorch tensors
# X_train = torch.from_numpy(train_feats).float().to(device)
# y_train = torch.from_numpy(train_labels).long().to(device)
# X_test  = torch.from_numpy(test_feats).float().to(device)
# y_test  = torch.from_numpy(test_labels).long().to(device)

# # Define simple linear model
# model = nn.Linear(N_CLUSTERS, 10).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)

# print('Training Softmax Regression...')
# for ep in range(1, EPOCHS+1):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train)
#     loss = criterion(outputs, y_train)
#     loss.backward()
#     optimizer.step()
#     if ep % 5 == 0:
#         print(f'Epoch {ep}/{EPOCHS}, loss={loss.item():.4f}')

In [ ]:
# # --- 9. Evaluate on test set ---
# model.eval()
# with torch.no_grad():
#     preds = model(X_test).argmax(dim=1)
#     acc = (preds == y_test).float().mean().item()
# print(f'Test Accuracy: {acc:.4f}')
# # Convert MPS tensor to CPU numpy for classification_report
# preds_np = preds.cpu().numpy()
# print(classification_report(test_labels, preds_np))

## SVM

In [ ]:
# 1. 定义一个 Pipeline，把 StandardScaler 和 LinearSVC 串起来
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc',    LinearSVC(dual=False, max_iter=500, random_state=42, verbose=2))
])

# 2. 指定要搜索的超参数网格
param_grid = {
    'svc__C':    [1],
    'svc__tol':  [1e-4]
}

# 3. 构造 GridSearchCV：5 折交叉验证，使用所有 CPU 核心
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    n_jobs=-1, # 使用所有 CPU 核心
    verbose=2, # 输出详细信息
    scoring='accuracy'
)

# 4. 在训练集上搜索最佳超参
grid.fit(train_feats, train_labels)

print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)



In [ ]:
# 5. 用最佳模型在测试集上评估
best_model = grid.best_estimator_
test_acc = best_model.score(test_feats, test_labels)
print(f"Test accuracy with best SVM: {test_acc:.4f}")

In [ ]:
# --- Randomly visualize classification results on test images ---
import matplotlib.pyplot as plt
# CIFAR-10 类别名称
class_names = testset.classes
# 随机选取几张图
for idx in random.sample(range(len(testset)), 10):
    img_t, label = testset[idx]
    # Tensor [C,H,W] 转为 HxWxC uint8 图像
    img_np = (img_t.numpy().transpose(1,2,0) * 255).astype(np.uint8)
    # 构建 BoVW 特征
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    kps = detect_interest_points(gray, method=INTEREST_METHOD)
    des = compute_custom_sift_descriptor(gray, kps)
    if des is None:
        hist = np.zeros(N_CLUSTERS, dtype=float)
    else:
        hist = build_bovw(des)
    # 标准化 & 预测
    # X = scaler.transform(hist.reshape(1, -1)) if 'scaler' in globals() else hist.reshape(1, -1)
    pred = best_model.predict(hist.reshape(1, -1))[0]
    # 可视化
    plt.figure(figsize=(4,4))
    plt.imshow(img_np)
    plt.title(f"GT: {class_names[label]}, Pred: {class_names[pred]}")
    plt.axis('off')
    plt.show()

In [ ]:
# # 定义 SVM 管道并开启 verbose 打印收敛信息，关闭 shrinking 提速
# svm_fast = make_pipeline(
#     StandardScaler(),
#     LinearSVC(
#         penalty='l2',
#         loss='squared_hinge',  # 改成 squared_hinge
#         dual=False,            # n_samples > n_features 时推荐 False
#         max_iter=5000,
#         tol=1e-4,
#         verbose=1
#     )
# )

# # 在 train_feats / train_labels 上训练
# svm_fast.fit(train_feats, train_labels)
# print("SVM training complete.")

In [ ]:
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# # 1. 预测
# svm_preds = svm_fast.predict(test_feats)

# # 2. 整体准确率
# acc = accuracy_score(test_labels, svm_preds)
# print(f"SVM Test Accuracy: {acc:.4f}")

# # 3. 详细报告
# print(classification_report(test_labels, svm_preds))

# # 4. （可选）混淆矩阵
# cm = confusion_matrix(test_labels, svm_preds)
# print("Confusion Matrix:")
# print(cm)